## A Hybrid Movie Recommendation System Project

### Business Problem and Stakeholder

A movie streaming platform is the primary stakeholder for this project. The platform offers a large catalog of movies and aims to improve user engagement, satisfaction, and retention by helping users quickly discover content that matches their preferences.

However, with thousands of available movies, users may struggle to identify titles they are likely to enjoy. Generic or poorly targeted recommendations can lead to user frustration, reduced platform usage, and increased churn. This challenge is further compounded by the cold-start problem, where new users or users with limited rating history receive less accurate recommendations.

The business problem addressed in this project is to develop a personalized movie recommendation system that can accurately suggest relevant movies to users based on their historical interactions, while also remaining effective when limited user rating data is available.

To address this challenge, the project implements a hybrid recommendation system that combines collaborative filtering based on user ratings with content-based filtering using user-generated movie tags. The system is designed to generate top-5 personalized movie recommendations that enhance the user experience and support the platform’s business objectives.

### Project Objectives
The objective of this project is to design and evaluate a hybrid movie recommendation system that provides personalized movie suggestions to users based on their historical preferences and content similarity.
Specifically, this project aims to:
- Build a collaborative filtering recommendation model using user–movie rating data to predict unseen ratings.
- Evaluate the collaborative filtering model using appropriate regression metrics, such as **RMSE** and **MAE**, to assess performance on unseen data.
- Develop a content-based recommendation component using user-generated movie tags to identify similar movies.
- Integrate collaborative filtering and content-based approaches into a hybrid recommendation system to address the cold-start problem.
- Generate and present **top-5 personalized movie recommendations** for users in a clear and interpretable manner.
- Demonstrate a clear modeling workflow, including data preparation, validation strategy, and result interpretation, suitable for a data science audience.


### Dataset Overview
This project uses the MovieLens (small) dataset, which contains user ratings, movie metadata, and user-generated tags. The dataset is commonly used for building and evaluating recommendation systems.
The following datasets are used in this project:
* ratings.csv: Contains user–movie interactions, including user IDs, movie IDs, ratings, and timestamps. This dataset forms the core input for training the collaborative filtering model.
* movies.csv: Contains movie titles and genre information associated with each movie ID. This dataset is used to display meaningful movie information in the final recommendations.
* tags.csv: Contains user-generated tags applied to movies. These tags are used to build a content-based recommendation component and to support a hybrid recommendation approach.
* links.csv: Contains external IMDb and TMDb identifiers and is not used in modeling.

In [3]:
# Loading the Datasets
import pandas as pd

ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")
tags = pd.read_csv("tags.csv")

### Preview the datasets

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


### Dataset Dimensions and Structure

In [7]:
print("Ratings shape:", ratings.shape)
ratings.info()

Ratings shape: (100836, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [8]:
print("Movies shape:", movies.shape)
movies.info()

Movies shape: (9742, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [9]:
print("Tags shape:", tags.shape)
tags.info()

Tags shape: (3683, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


### Validation Strategy

To evaluate the performance of the recommendation system and ensure that the model generalizes well to unseen data, a train–test split validation strategy is employed.

For the collaborative filtering component, the user–movie ratings data is divided into a training set and a test set. The model is trained exclusively on the training data, while performance is evaluated on the test data using ratings that were not seen during training. This approach helps prevent overfitting and provides a realistic estimate of how the model would perform in a real-world setting.

Since the recommendation task involves predicting numerical ratings, regression-based evaluation metrics are used. Specifically, Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE) are used to measure the difference between predicted and actual user ratings on the test set.

For the content-based and hybrid recommendation components, quantitative evaluation is less straightforward due to the lack of explicit ground-truth labels for similarity-based recommendations. Therefore, these components are evaluated qualitatively by inspecting the relevance and coherence of the generated movie recommendations, particularly in cold-start scenarios where user rating history is limited.

This validation strategy ensures that the collaborative filtering model is evaluated rigorously, while the hybrid approach is assessed in a manner consistent with its intended real-world application

### Data Preprocessing and Cleaning
Before building the recommendation models, the datasets are cleaned and prepared to ensure consistency and reliability. Since the MovieLens dataset is well-structured, preprocessing focuses primarily on validating data quality and preparing inputs for collaborative filtering and content-based modeling.

1. Preprocessing the Ratings Data

The ratings dataset is inspected to confirm that all ratings are valid, user and movie identifiers are consistent, and no missing values are present.

In [10]:
# Check for missing values in ratings
ratings.isna().sum()


userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [11]:
# Confirm rating scale
ratings['rating'].unique()


array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

In [12]:
# Drop timestamp (not required for modeling)
ratings = ratings.drop(columns=['timestamp'])


2. Preprocessing the Movies Data

The movies dataset is used for mapping movie IDs to titles and genres.

In [13]:
# Check for missing values
movies.isna().sum()


movieId    0
title      0
genres     0
dtype: int64

In [14]:
# Ensure movieId uniqueness
movies['movieId'].is_unique


True

3. Preprocessing the Tags Data

Tags are used to build the content-based recommendation component.

In [15]:
# Check for missing values
tags.isna().sum()


userId       0
movieId      0
tag          0
timestamp    0
dtype: int64

In [16]:
# Convert tags to lowercase for consistency
tags['tag'] = tags['tag'].str.lower()


In [17]:
# Remove timestamp column
tags = tags.drop(columns=['timestamp'])


4. Aligning Movie IDs Across Datasets

To ensure consistency across datasets, only movies that appear in the ratings data are retained in the movies and tags datasets.

In [18]:
valid_movie_ids = set(ratings['movieId'])

movies = movies[movies['movieId'].isin(valid_movie_ids)]
tags = tags[tags['movieId'].isin(valid_movie_ids)]


5. Preparing Tags for Content-Based Modeling

Tags are aggregated at the movie level to create a single text representation per movie.

In [19]:
# Combine tags per movie
movie_tags = (
    tags.groupby('movieId')['tag']
    .apply(lambda x: ' '.join(x))
    .reset_index()
)

movie_tags.head()


,movieId,tag
0,1,pixar pixar fun
1,2,fantasy magic board game robin williams game
2,3,moldy old
3,5,pregnancy remake
4,7,remake


### Collaborative Filtering Using SVD

In this section, a collaborative filtering recommendation model is developed using Singular Value Decomposition (SVD). Collaborative filtering leverages historical user–movie rating interactions to learn underlying patterns in user preferences and item characteristics.

The SVD algorithm factorizes the user–item rating matrix into lower-dimensional latent factors, allowing the model to estimate missing ratings for user–movie pairs that were not observed during training. These predicted ratings are then used to generate personalized movie recommendations.

To evaluate the model’s ability to generalize to unseen data, the ratings dataset is split into training and test sets. Model performance is assessed using Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE), which measure the difference between predicted and actual ratings. This evaluation provides a quantitative basis for assessing the effectiveness of the collaborative filtering approach.

The trained SVD model serves as the primary recommendation engine in this project and forms the foundation for generating top-5 personalized movie recommendations

In [20]:
# Import necessary libraries
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy


In [21]:
# Prepare the Data for Surprise
# Define the rating scale
reader = Reader(rating_scale=(0.5, 5.0))

# Load data into Surprise format
data = Dataset.load_from_df(
    ratings[['userId', 'movieId', 'rating']],
    reader
)


In [23]:
# Train–Test Split
# Split data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


In [25]:
# Train the SVD Model
# Initialize the SVD model
svd_model = SVD(random_state=42)

# Train the model
svd_model.fit(trainset)
# At this point, the model learns: User preferences, Movie characteristics, Hidden patterns (latent factors)


In [28]:
# Evaluate the Model
# Make predictions on the test set
predictions = svd_model.test(testset)

# Evaluate performance
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
print(f"RMSE: {rmse}, MAE: {mae}")

# What these metrics mean:
   # RMSE → penalizes large errors more
   # MAE → average prediction error
   # Lower = better

# This satisfies quantitative validation requirement.


RMSE: 0.8807
MAE:  0.6766
RMSE: 0.8807462819979623, MAE: 0.6765729095860605


The SVD model achieves reasonable RMSE and MAE scores, indicating that it can effectively learn user preferences and predict unseen ratings. These results suggest the model generalizes well to new user–movie interactions

### Generating Top-5 Movie Recommendations

After training and evaluating the collaborative filtering model, the next step is to use the trained SVD model to generate personalized movie recommendations. The goal is to recommend movies that a user is likely to enjoy but has not yet rated.

For a selected user, the model predicts ratings for all movies that the user has not previously interacted with. These predicted ratings are then ranked from highest to lowest, and the top five movies with the highest predicted ratings are selected as recommendations.

To ensure the recommendations are meaningful and interpretable, movies that the user has already rated are excluded from the recommendation list. Movie identifiers are mapped to their corresponding titles and genres to present the results in a user-friendly format.

The resulting top-5 recommendations demonstrate how the collaborative filtering model can be applied in practice to support personalized content discovery on a movie streaming platform

In [29]:
import numpy as np
import pandas as pd

def get_top_n_recommendations(user_id, ratings_df, movies_df, model, n=5):
    """
    Returns top-n movie recommendations for a given user_id using a trained Surprise model.
    Excludes movies the user has already rated.
    """
    # All movie IDs in the dataset
    all_movie_ids = movies_df['movieId'].unique()
    
    # Movies already rated by this user
    rated_movie_ids = ratings_df.loc[ratings_df['userId'] == user_id, 'movieId'].unique()
    
    # Movies not yet rated
    unseen_movie_ids = np.setdiff1d(all_movie_ids, rated_movie_ids)
    
    # Predict ratings for unseen movies
    preds = []
    for movie_id in unseen_movie_ids:
        est = model.predict(user_id, movie_id).est
        preds.append((movie_id, est))
    
    # Sort by predicted rating (highest first)
    preds.sort(key=lambda x: x[1], reverse=True)
    
    # Top-N results
    top_n = preds[:n]
    
    # Convert to a clean DataFrame with titles
    top_n_df = pd.DataFrame(top_n, columns=['movieId', 'predicted_rating'])
    top_n_df = top_n_df.merge(movies_df[['movieId', 'title', 'genres']], on='movieId', how='left')
    
    return top_n_df[['movieId', 'title', 'genres', 'predicted_rating']]


In [32]:
# Pick a user and get Top-5 recommendations
# Choose a user that definitely exists
example_user = ratings['userId'].iloc[0]

top5 = get_top_n_recommendations(
    example_user,
    ratings,
    movies,
    svd_model,
    n=5
)

top5

# It is important to note that the relative ranking of recommendations is more critical than the exact predicted rating values.



,movieId,title,genres,predicted_rating
0,541,Blade Runner (1982),Action|Sci-Fi|Thriller,5.0
1,741,Ghost in the Shell (Kôkaku kidôtai) (1995),Animation|Sci-Fi,5.0
2,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,5.0
3,908,North by Northwest (1959),Action|Adventure|Mystery|Romance|Thriller,5.0
4,912,Casablanca (1942),Drama|Romance,5.0


### Interpretation of Top-5 Recommendations

The top-5 recommended movies produced by the collaborative filtering model consist of widely acclaimed and highly rated films across multiple genres, including science fiction, drama, action, and romance. The consistently high predicted ratings suggest that the SVD model has identified strong alignment between the selected user’s historical preferences and the latent characteristics of these movies.

While all recommended movies receive the maximum predicted rating, this behavior is expected in recommendation systems where the primary objective is to rank items by relevance rather than to provide perfectly calibrated rating estimates. The results demonstrate that the model is able to surface high-quality and diverse movie recommendations, supporting its effectiveness as a personalized recommendation engine.


## Content-Based Filtering Using TF-IDF on Tags

In this section, a content-based recommendation component is built using user-generated movie tags. Tags provide descriptive text about movies (e.g., “sci-fi”, “classic”, “thriller”), which can be used to measure similarity between movies.

To represent movie tags numerically, we apply **TF-IDF (Term Frequency–Inverse Document Frequency)** to convert the aggregated tags for each movie into feature vectors. We then compute **cosine similarity** between movie vectors to identify movies with similar tag profiles.

This component supports the hybrid recommendation approach by providing meaningful recommendations in situations where collaborative filtering may struggle, such as the cold-start case (new users with limited rating history).


In [33]:
# Aggregate tags per movie into a single text field
movie_tags = (
    tags.groupby('movieId')['tag']
    .apply(lambda x: ' '.join(x.astype(str)))
    .reset_index()
)
movie_tags.head()


,movieId,tag
0,1,pixar pixar fun
1,2,fantasy magic board game robin williams game
2,3,moldy old
3,5,pregnancy remake
4,7,remake


In [34]:
# Merge tags into the movies table
movies_with_tags = movies.merge(movie_tags, on='movieId', how='left')

# Fill missing tag text
movies_with_tags['tag'] = movies_with_tags['tag'].fillna('')

movies_with_tags[['movieId', 'title', 'tag']].head()


,movieId,title,tag
0,1,Toy Story (1995),pixar pixar fun
1,2,Jumanji (1995),fantasy magic board game robin williams game
2,3,Grumpier Old Men (1995),moldy old
3,4,Waiting to Exhale (1995),
4,5,Father of the Bride Part II (1995),pregnancy remake


In [36]:
# Vectorize the tags using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    stop_words='english',
    min_df=2  # ignore extremely rare tags for stability
)

tfidf_matrix = tfidf.fit_transform(movies_with_tags['tag'])
tfidf_matrix.shape
# tfidf_matrix is now our content feature representation for each movie.

(9724, 722)

In [37]:
# Cosine similarity (movie-to-movie)
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape


(9724, 9724)

In [38]:
# A content-based “similar movies” function (Top-N). This recommends movies similar to a given movie using tag similarity
# Create a mapping: movieId -> row index in movies_with_tags
movieid_to_index = pd.Series(movies_with_tags.index, index=movies_with_tags['movieId']).to_dict()

def recommend_similar_movies(movie_id, movies_df, cosine_sim_matrix, movieid_to_idx, n=5):
    """
    Recommend top-n movies similar to a given movie_id using cosine similarity on TF-IDF tag vectors.
    """
    if movie_id not in movieid_to_idx:
        return pd.DataFrame(columns=['movieId', 'title', 'genres', 'similarity_score'])
    
    idx = movieid_to_idx[movie_id]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    
    # Sort by similarity (exclude itself at index 0 after sorting)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = [s for s in sim_scores if s[0] != idx][:n]
    
    # Build results
    rec_indices = [i for i, _ in sim_scores]
    rec_scores = [score for _, score in sim_scores]
    
    recs = movies_df.iloc[rec_indices][['movieId', 'title', 'genres']].copy()
    recs['similarity_score'] = rec_scores
    
    return recs


In [39]:
# Picks a movie from Top-5 SVD output, e.g., Blade Runner (1982) has movieId 541
recommend_similar_movies(541, movies_with_tags, cosine_sim, movieid_to_index, n=5)


,movieId,title,genres,similarity_score
9629,180031,The Shape of Water (2017),Adventure|Drama|Fantasy,0.447116
938,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller,0.399463
9586,176371,Blade Runner 2049 (2017),Sci-Fi,0.393653
8066,99917,Upstream Color (2013),Romance|Sci-Fi|Thriller,0.342471
3281,4446,Final Fantasy: The Spirits Within (2001),Adventure|Animation|Fantasy|Sci-Fi,0.314671


## Hybrid Recommendation Strategy

To address limitations inherent in using a single recommendation approach, a hybrid strategy is implemented that combines collaborative filtering and content-based filtering.

For users with sufficient rating history, collaborative filtering using SVD is employed as the primary recommendation method due to its ability to capture collective user preferences. For users with limited interaction history, a content-based approach using TF-IDF similarity on movie tags is used to generate recommendations based on movies the user has rated highly.

This hybrid approach improves recommendation robustness and helps mitigate the cold-start problem, while maintaining interpretability and computational efficiency.


In [40]:
# Hybrid Recommendation Function
def hybrid_recommendations(
    user_id,
    ratings_df,
    movies_df,
    svd_model,
    cosine_sim_matrix,
    movieid_to_idx,
    n=5,
    min_ratings=5
):
    # User rating history
    user_ratings = ratings_df[ratings_df['userId'] == user_id]
    
    # Case 1: Enough history → use SVD
    if len(user_ratings) >= min_ratings:
        svd_recs = get_top_n_recommendations(
            user_id, ratings_df, movies_df, svd_model, n=n
        )
        return svd_recs.assign(source='Collaborative Filtering (SVD)')
    
    # Case 2: Cold-start → use content-based
    else:
        # Use the user's highest-rated movie as seed
        top_movie_id = user_ratings.sort_values(
            'rating', ascending=False
        ).iloc[0]['movieId']
        
        content_recs = recommend_similar_movies(
            top_movie_id,
            movies_df,
            cosine_sim_matrix,
            movieid_to_idx,
            n=n
        )
        return content_recs.assign(source='Content-Based (TF-IDF)')


### Cold-Start Test (Hybrid System)

To test the hybrid recommendation strategy under a cold-start scenario, a user with fewer than five ratings was selected. Since collaborative filtering requires sufficient interaction history to generate reliable predictions, the hybrid system falls back to the content-based component for this user.

The content-based recommender uses TF-IDF representations of movie tags and cosine similarity to recommend movies that are most similar to the user’s highest-rated movie. This test demonstrates that the system can still produce relevant recommendations even when user rating history is limited.


In [42]:
# Find a cold-start user (few ratings).
# A cold-start user is someone with very few ratings (e.g., 1–4). Since SVD needs enough history, we switch to content-based recommendations
# Count ratings per user
user_rating_counts = ratings.groupby('userId').size().reset_index(name='n_ratings')

# Find users with few ratings (cold-start candidates)
cold_users = user_rating_counts[user_rating_counts['n_ratings'] < 50].sort_values('n_ratings')
cold_users.head(10)


,userId,n_ratings
405,406,20
430,431,20
188,189,20
146,147,20
206,207,20
52,53,20
193,194,20
277,278,20
319,320,20
568,569,20


In [44]:
# This helps you see what the user actually liked (seed movie choice)
# Select a cold-start user and inspect their ratings
cold_user_id = cold_users.iloc[0]['userId']
cold_user_id
# Show this user's ratings (with titles)
cold_user_ratings = ratings[ratings['userId'] == cold_user_id].merge(
    movies[['movieId', 'title', 'genres']], on='movieId', how='left'
).sort_values('rating', ascending=False)

cold_user_ratings


,userId,movieId,rating,title,genres
19,406,56949,5.0,27 Dresses (2008),Comedy|Romance
16,406,33669,5.0,"Sisterhood of the Traveling Pants, The (2005)",Adventure|Comedy|Drama
15,406,5620,5.0,Sweet Home Alabama (2002),Comedy|Romance
11,406,2125,5.0,Ever After: A Cinderella Story (1998),Comedy|Drama|Romance
1,406,261,4.0,Little Women (1994),Drama
18,406,46972,4.0,Night at the Museum (2006),Action|Comedy|Fantasy|IMAX
0,406,135,4.0,Down Periscope (1996),Comedy
5,406,1022,3.5,Cinderella (1950),Animation|Children|Fantasy|Musical|Romance
13,406,2694,3.5,Big Daddy (1999),Comedy
14,406,2722,3.5,Deep Blue Sea (1999),Action|Horror|Sci-Fi|Thriller


In [46]:
# hybrid system for the cold-start user
hybrid_recs = hybrid_recommendations(
    user_id=cold_user_id,
    ratings_df=ratings,
    movies_df=movies_with_tags,   # IMPORTANT: use the dataframe that includes tags
    svd_model=svd_model,
    cosine_sim_matrix=cosine_sim,
    movieid_to_idx=movieid_to_index,
    n=5,
    min_ratings=5
)

hybrid_recs


,movieId,title,genres,predicted_rating,source
0,6016,City of God (Cidade de Deus) (2002),Action|Adventure|Crime|Drama|Thriller,4.471538,Collaborative Filtering (SVD)
1,904,Rear Window (1954),Mystery|Thriller,4.441445,Collaborative Filtering (SVD)
2,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,4.392330,Collaborative Filtering (SVD)
3,1252,Chinatown (1974),Crime|Film-Noir|Mystery|Thriller,4.383121,Collaborative Filtering (SVD)
4,1204,Lawrence of Arabia (1962),Adventure|Drama|War,4.379617,Collaborative Filtering (SVD)
